# Символьное дифференцирование

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 18 ноября 10:00**

Символьное дифференцирование это инструмент для автоматического вывода формул производных, который открывает возможности для анализа сложных функций, оптимизации процессов и работы с уравнениями. Мы уже на многих занятиях сталкивались с этой темой - давайте попробуем реализовать собственное!

## Выражение

Создадим основной класс `Expr`, от которого будут наследоваться различные типы выражений, такие как константы, переменные, суммы, произведения и другие. Класс должен содержать методы:
* `__call__`, который будет вычислять значение выражения, используя переданный ему контекст (словарь, связывающий имена переменных с их значениями).
* `d`, принимающий имя переменной, по которой требуется вычислить производную, и возвращающий выражение, представляющее производную по этой переменной.

Эти методы нужно будет переопределить в каждом из подклассов для корректного выполнения операций.

In [2]:
class Expr:
    def __call__(self, **context):
        raise NotImplementedError("Метод __call__ должен быть реализован в подклассе.")
        pass
    
    def d(self, wrt):
        raise NotImplementedError("Метод d должен быть реализован в подклассе.")
        pass

Создайте классы для двух видов выражений: `Const`, представляющий константу, и` Var`, представляющий переменную. Чтобы упростить использование, вместо обращения к конструкторам этих классов, будем использовать их однобуквенные сокращённые обозначения.

**Пример использования:**
```python
V = Var
C = Const

C(5)()
5
C(5).d(V("x"))()
0
V("x")(x=5)
5
V("x").d(V("y"))(x=5)
0
V("x").d(V("x"))(x=5)
1
```

In [5]:
class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        return Const(0)  # Производная константы всегда 0
    pass

class Var(Expr):
    def __init__(self, name):
        self.name = name
    
    def __call__(self, **context):
        return context.get(self.name, 0)  # Возвращаем значение переменной из context или 0, если переменная не найдена
    
    def d(self, wrt):
        if self.name == wrt.name:
            return Const(1)  # Производная переменной по самой себе равна 1
        else:
            return Const(0)  # Производная по другой переменной равна 0
    pass

In [7]:
V = Var
C = Const

In [13]:
print(C(5)())
print(C(5).d(V("x"))())
print(V("x")(x=5))
print(V("x").d(V("y"))(x=5))
print(V("x").d(V("x"))(x=5))

5
0
5
0
1


## Бинарные операции

Создайте классы для бинарных операций: `Sum`, `Product` и `Fraction`. Поскольку бинарные операции определяются двумя операндами, их конструктор будет одинаковым для всех этих классов. Поэтому его можно вынести в отдельный базовый класс, чтобы избежать дублирования кода.

In [17]:
class BinOp(Expr):
    def __init__(self, expr1, expr2):
        self.expr1, self.expr2 = expr1, expr2

Реализуйте `Sum` для суммирования, `Product` для умножения и `Fraction` для деления.

**Пример использования:**

```python
Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5)
7.0
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2)
3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2)
-3.5
Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2)
-1.25
```

In [19]:
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)
    
    def d(self, wrt):
        # Производная суммы: d(f + g) = f' + g'
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))
    pass

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        # Производная произведения: d(f * g) = f' * g + f * g'
        return Sum(Product(self.expr1.d(wrt), self.expr2), Product(self.expr1, self.expr2.d(wrt)))
    pass

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        # Производная дроби: d(f / g) = (f' * g - f * g') / g^2
        numerator = Sum(Product(self.expr1.d(wrt), self.expr2), Product(Const(-1), Product(self.expr1, self.expr2.d(wrt)))) # числитель
        denominator = Product(self.expr2, self.expr2) # знаменатель
        return Fraction(numerator, denominator)
    pass

In [21]:
print(Sum(V("x"), Fraction(V("x"), V("y")))(x=5, y=2.5))
print(Fraction(Sum(C(5), V("y")), Product(V("x"), V("y")))(x=1, y=2))
print(Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("x"))(x=1, y=2))
print(Fraction(Sum(C(5), V("y")), Product(V("x"), V("y"))).d(V("y"))(x=1, y=2))

7.0
3.5
-3.5
-1.25


## Перегрузка операторов

Добавьте перегрузку операторов в базовых класс `Expr`. Обратите что в классах мы можем тоже заменить на использование операторов.
```python  
-e         e.__neg__()
+e         e.__pos__()
e1 + e2    e1.__add__(e2)
e1 - e2    e1.__sub__(e2)
e1 * e2    e1.__mul__(e2)
e1 / e2    e1.__truediv__(e2)
```

**Пример использования:**

```python
(V("x") * V("x") / V("y"))(x=5, y=2.5)
10.0
```

In [35]:
# класс с перегрузкой операторов
class Expr:
    def __call__(self, **context):
        raise NotImplementedError("Метод __call__ должен быть реализован в подклассе.")
    
    def d(self, wrt):
        raise NotImplementedError("Метод d должен быть реализован в подклассе.")
    
    # Перегрузка операторов
    def __neg__(self):
        return Product(Const(-1), self)  # -e -> (-1) * e
    
    def __pos__(self):
        return self  # +e -> e
    
    def __add__(self, other):
        return Sum(self, other)  # e1 + e2
    
    def __sub__(self, other):
        return Sum(self, Product(Const(-1), other))  # e1 - e2 -> e1 + (-1) * e2
    
    def __mul__(self, other):
        return Product(self, other)  # e1 * e2
    
    def __truediv__(self, other):
        return Fraction(self, other)  # e1 / e2

In [37]:
class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        return Const(0)  # Производная константы всегда 0
    pass

class Var(Expr):
    def __init__(self, name):
        self.name = name
    
    def __call__(self, **context):
        return context.get(self.name, 0)  # Возвращаем значение переменной из context или 0, если переменная не найдена
    
    def d(self, wrt):
        if self.name == wrt.name:
            return Const(1)  # Производная переменной по самой себе равна 1
        else:
            return Const(0)  # Производная по другой переменной равна 0
    pass

In [39]:
class BinOp(Expr):
    def __init__(self, expr1, expr2):
        self.expr1, self.expr2 = expr1, expr2

In [41]:
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)
    
    def d(self, wrt):
        # Производная суммы: d(f + g) = f' + g'
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))
    pass

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        # Производная произведения: d(f * g) = f' * g + f * g'
        return Sum(Product(self.expr1.d(wrt), self.expr2), Product(self.expr1, self.expr2.d(wrt)))
    pass

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        # Производная дроби: d(f / g) = (f' * g - f * g') / g^2
        numerator = Sum(Product(self.expr1.d(wrt), self.expr2), Product(Const(-1), Product(self.expr1, self.expr2.d(wrt)))) # числитель
        denominator = Product(self.expr2, self.expr2) # знаменатель
        return Fraction(numerator, denominator)
    pass

In [43]:
V = Var
C = Const

In [45]:
print((V("x") * V("x") / V("y"))(x=5, y=2.5))

10.0


## Метод Ньютона-Рафсона

Напишите функцию `newton_raphson`, которая принимает дифференцируемую функцию  $f$  от переменной  $x$ , начальное приближение  $x_0$ , и положительное число  $\epsilon$ , задающее точность вычислений. Функция должна возвращать значение  $x$ , при котором  $f(x)$  становится равным нулю. Метод Ньютона-Рафсона выполняет итеративный поиск корня функции  $f(x)$ , начиная с начального значения  $x_0$ , и использует правило  
$$x_{n+1} = x_n - \frac{f(x_n)}{f{\prime}(x_n)}$$  
для обновления  $x$  на каждом шаге. Итерации продолжаются до тех пор, пока условие остановки  $|x_{n+1} - x_n| \leq \epsilon$  не будет выполнено.

**Пример использования:**

```python
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
zero, f(x=zero)
(1.000000000001132, -2.490496697760136e-11)
```

In [48]:
def newton_raphson(expr, x0, eps=1e-4):
    x_n = x0
    while True:
        # Вычисляем f(x_n) и f'(x_n)
        f_xn = expr(x=x_n)
        f_prime_xn = expr.d(Var("x"))(x=x_n)
        
        # Проверяем, чтобы производная не была равна нулю (деление на ноль невозможно)
        if f_prime_xn == 0:
            raise ValueError("Производная равна нулю, метод Ньютона-Рафсона не применим.")
        
        # Обновляем x по формуле метода Ньютона-Рафсона
        x_next = x_n - f_xn / f_prime_xn
        
        # Условие выхода: если разница между текущим и следующим значением меньше eps
        if abs(x_next - x_n) <= eps:
            return x_next
        
        # Переход к следующей итерации
        x_n = x_next
    pass

In [50]:
x = Var("x")
f = Const(-5) * x * x * x * x * x + Const(3) * x + Const(2)
zero = newton_raphson(f, 0.5, eps=1e-4)
print(zero, f(x=zero))

1.0000000000000653 -1.4384049507043528e-12
